<a href="https://colab.research.google.com/github/pallavgoyal136/Python_Programming_Assignment/blob/main/Python_Programming_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
training_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

In [6]:
def convert_units(value):
  if ' Hund+' in value:
    return int(value.replace(' Hund+', '')) * 10**2
  if ' Thou+' in value:
    return int(value.replace(' Thou+', '')) * 10**3
  elif ' Lac+' in value:
    return int(value.replace(' Lac+', '')) * 10**5
  elif ' Crore+' in value:
    return int(value.replace(' Crore+', '')) * 10**7
  else:
    return int(value)

training_data['Total Assets'] = training_data['Total Assets'].apply(convert_units)
training_data['Liabilities'] = training_data['Liabilities'].apply(convert_units)

test_data['Total Assets'] = test_data['Total Assets'].apply(convert_units)
test_data['Liabilities'] = test_data['Liabilities'].apply(convert_units)

df=training_data
df2=test_data

column_name='Total Assets'
df[column_name] = np.log10(df[column_name]+1)
df2[column_name] = np.log10(df2[column_name]+1)

column_name='Liabilities'
df[column_name] = np.log10(df[column_name]+1)
df2[column_name] = np.log10(df2[column_name]+1)

In [7]:
def extract_category(name):
    if name.startswith(('Dr.', 'Adv.')):
        return name[0:2]
    else:
        return 'None'

df['Candidate']=df['Candidate'].apply(extract_category)
one_hot_encoded = pd.get_dummies(df['Candidate'], dtype=int)
df=pd.concat([df, one_hot_encoded], axis=1)
df2['Candidate']=df2['Candidate'].apply(extract_category)
one_hot_encoded = pd.get_dummies(df2['Candidate'], dtype=int)
df2=pd.concat([df2, one_hot_encoded], axis=1)

In [8]:
one_hot_encoded = pd.get_dummies(training_data['Party'], dtype=int)
df = pd.concat([df, one_hot_encoded], axis=1)

one_hot_encoded = pd.get_dummies(test_data['Party'], dtype=int)
df2 = pd.concat([df2, one_hot_encoded], axis=1)

one_hot_encoded = pd.get_dummies(training_data['state'], dtype=int)
df = pd.concat([df, one_hot_encoded], axis=1)

one_hot_encoded = pd.get_dummies(test_data['state'], dtype=int)
df2 = pd.concat([df2, one_hot_encoded], axis=1)

columns_to_drop = ['Liabilities','Candidate', 'state', 'Party', 'Constituency ∇', 'ID','None', 'HARYANA', 'HIMACHAL PRADESH', 'MAHARASHTRA', 'PUDUCHERRY', 'PUNJAB', 'TRIPURA', 'RJD', 'SP']
columns_to_drop2 = ['Liabilities','Candidate', 'state', 'Party', 'Constituency ∇', 'ID','None','HARYANA', 'HIMACHAL PRADESH', 'MAHARASHTRA', 'PUDUCHERRY', 'PUNJAB', 'TRIPURA','RJD', 'SP']
df.drop(columns_to_drop, axis=1, inplace=True)
df2.drop(columns_to_drop2, axis=1, inplace=True)

In [9]:
import sklearn.naive_bayes as naive_bayes

y_train = df['Education']
X_train = df.drop(columns=['Education'])
X_test = df2

model = naive_bayes.BernoulliNB()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
prediction=pd.DataFrame({'ID':test_data['ID'],'Education': y_pred})
prediction.to_csv('/content/prediction.csv',index=False)